In [14]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torchsummary import *
from gman import *
import time
import datetime
os.environ['CUDA_VISIBLE_DEVICES']='0'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
print(f"Using device {device}")

Using device cuda:0


# Data

In [15]:
# log string
def log_string(log, string):
    log.write(string + '\n')
    log.flush()
    print(string)
# metric
def metric(pred, label):
    mask = torch.ne(label, 0)
    mask = mask.type(torch.float32)
    mask /= torch.mean(mask)
    mae = torch.abs(torch.sub(pred, label)).type(torch.float32)
    rmse = mae ** 2
    mape = mae / label
    mae = torch.mean(mae)
    rmse = rmse * mask
    rmse = torch.sqrt(torch.mean(rmse))
    mape = mape * mask
    mape = torch.mean(mape)
    return mae, rmse, mape

def seq2instance(data, P, Q):
    num_step, dims = data.shape
    num_sample = num_step - P - Q + 1
    x = torch.zeros(num_sample, P, dims)
    y = torch.zeros(num_sample, Q, dims)
    for i in range(num_sample):
        x[i] = data[i : i + P]
        y[i] = data[i + P : i + P + Q]
    return x, y

def loadData(data_args):
    # Traffic
    df = pd.read_csv(data_args['data_file'],index_col=0)
    Traffic = torch.from_numpy(df.values)
    # train/val/test 
    num_step = df.shape[0]
    train_steps = round(data_args['train_ratio'] * num_step)
    test_steps = round(data_args['test_ratio'] * num_step)
    val_steps = num_step - train_steps - test_steps
    train = Traffic[: train_steps]
    val = Traffic[train_steps : train_steps + val_steps]
    test = Traffic[-test_steps :]
    # X, Y 
    trainX, trainY = seq2instance(train, data_args['P'], data_args['Q'])
    valX, valY = seq2instance(val,data_args['P'], data_args['Q'])
    testX, testY = seq2instance(test,data_args['P'], data_args['Q'])
    # normalization
    mean, std = torch.mean(trainX), torch.std(trainX)
    trainX = (trainX - mean) / std
    valX = (valX - mean) / std
    testX = (testX - mean) / std
    
    # temporal embedding 
    time = pd.DatetimeIndex(df.index)
    dayofweek = torch.reshape(torch.tensor(time.weekday), (-1, 1))
#     timeofday = (time.hour * 3600 + time.minute * 60 + time.second) \
#                 // time.freq.delta.total_seconds()
#     timeofday = torch.reshape(torch.tensor(timeofday), (-1, 1))
    timeofday = (time.values - df.index.values.astype("datetime64[D]")) / np.timedelta64(1, "D")
    timeofday = torch.reshape(torch.tensor(timeofday), (-1, 1))        
    time = torch.cat((dayofweek, timeofday), -1)
    # train/val/test
    train = time[: train_steps]
    val = time[train_steps : train_steps + val_steps]
    test = time[-test_steps :]
    # shape = (num_sample, P + Q, 2)
    trainTE = seq2instance(train,data_args['P'], data_args['Q'])
    trainTE = torch.cat(trainTE, 1).type(torch.int32)
    valTE = seq2instance(val, data_args['P'], data_args['Q'])
    valTE = torch.cat(valTE, 1).type(torch.int32)
    testTE = seq2instance(test, data_args['P'],data_args['Q'])
    testTE = torch.cat(testTE, 1).type(torch.int32)
    
    return (trainX, trainTE, trainY, valX, valTE, valY, testX, testTE, testY,
             mean, std)

In [23]:
data_args={
'data_file' : '../Wind/tj_wind_10m.csv',
# 'SE_file' : './Dataset/PEMS-BAY/SE(PeMS).txt',
'train_ratio' : 0.7,
'test_ratio' : 0.1,
'P' : 32,
'Q' : 8,
}
(trainX, trainTE, trainY, valX, valTE, valY, testX, testTE, testY,
 mean, std) = loadData(data_args)
SE = torch.zeros((12,64))

In [8]:
# adj = pd.read_csv('./Dataset/PEMS-BAY/Adj(PeMS).txt',header =None)
# adj = adj.iloc[:,0].apply(lambda x: x.split(' ')[-1])
# adj = np.array(list(map(float,adj.values))).reshape((325,325))
# adj = torch.from_numpy(adj)
# adj = adj [0:,0:]

# Model

In [20]:
nnodes = 30
ne = 3
ndim = 32
alpha = 0.5
L = 3
K = 8
d = 8
num_his = data_args['P']
num_pred = data_args['Q']
bn_decay =0.1
steps_per_day = 288
use_bias = False
mask = True
model = GMAN(L,K,d,num_his,bn_decay,steps_per_day,use_bias,mask,SE,device).to(device)

In [10]:
# from torchsummary import *
# summary(model,input_size=[(12,325),(15,2)],batch_size=32)

# Train

In [24]:
max_epochs =100
patience =7
batch_size = 64
LR = 0.01
decay_epoch = 10
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                      step_size=decay_epoch,
                                      gamma=0.9)
loss_criterion = nn.L1Loss().to(device)
model_file = '../result/model/gman_tj10_32-8.pkl'

In [21]:
log_file = '../Wind/log(bjWind)'
log = open(log_file, 'w')
log_string(log, '**** training model ****')
num_val = valX.shape[0]
num_train = trainX.shape[0]
# num_train = 800
num_test = testX.shape[0]
train_num_batch = math.ceil(num_train / batch_size)
val_num_batch = math.ceil(num_val / batch_size)
test_num_batch = math.ceil(num_test / batch_size)
wait = 0
val_loss_min = float('inf')
best_model_wts = None
train_total_loss = []
val_total_loss = []
# shuffle
SE = SE.to(device)
# model = torch.load(model_file)
for epoch in range(max_epochs):    
    if wait >= patience:
        log_string(log, f'early stop at epoch: {epoch:04d}')
        break
    permutation = torch.randperm(num_train)
    trainX = trainX[permutation]
    trainTE = trainTE[permutation]
    trainY = trainY[permutation]
    # train
    start_train = time.time()
    model.train()
    train_loss = 0
    for batch_idx in range(train_num_batch):
        start_idx = batch_idx * batch_size
        end_idx = min(num_train, (batch_idx + 1) * batch_size)
        X = trainX[start_idx: end_idx].to(device)
        TE = trainTE[start_idx: end_idx].to(device)
        label = trainY[start_idx: end_idx].to(device)
        optimizer.zero_grad()
        pred = model(X, TE)
        pred = pred * std + mean
        loss_batch = loss_criterion(pred, label)
        train_loss += float(loss_batch) * (end_idx - start_idx)
        loss_batch.backward()
        optimizer.step()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        if (batch_idx+1) % 5 == 0:
            print(f'Training batch: {batch_idx+1} in epoch:{epoch}, training batch loss:{loss_batch:.4f}')
        del X, TE, label, pred, loss_batch
    train_loss /= num_train
    train_total_loss.append(train_loss)
    end_train = time.time()

    # val loss
    start_val = time.time()
    val_loss = 0
    model.eval()
    with torch.no_grad():
        for batch_idx in range(val_num_batch):
            start_idx = batch_idx * batch_size
            end_idx = min(num_val, (batch_idx + 1) * batch_size)
            X = valX[start_idx: end_idx].to(device)
            TE = valTE[start_idx: end_idx].to(device)
            label = valY[start_idx: end_idx].to(device)
            pred = model(X, TE)
            pred = pred * std + mean
            loss_batch = loss_criterion(pred, label)
            val_loss += loss_batch * (end_idx - start_idx)
            del X, TE, label, pred, loss_batch
    val_loss /= num_val
    val_total_loss.append(val_loss)
    end_val = time.time()
    log_string(
        log,
        '%s | epoch: %04d/%d, training time: %.1fs, inference time: %.1fs' %
        (datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'), epoch + 1,
         max_epochs, end_train - start_train, end_val - start_val))
    log_string(
        log, f'train loss: {train_loss:.4f}, val_loss: {val_loss:.4f}')
    if val_loss <= val_loss_min:
        log_string(
            log,
            f'val loss decrease from {val_loss_min:.4f} to {val_loss:.4f}, saving model to {model_file}')
        wait = 0
        val_loss_min = val_loss
        best_model_wts = model.state_dict()
        torch.save(model, model_file)
    else:
        wait += 1
    scheduler.step()
model.load_state_dict(best_model_wts)
torch.save(model, model_file)
log_string(log, f'Training and validation are completed, and model has been stored as {model_file}')

**** training model ****
Training batch: 5 in epoch:0, training batch loss:0.9184
Training batch: 10 in epoch:0, training batch loss:0.9113
Training batch: 15 in epoch:0, training batch loss:0.9126
Training batch: 20 in epoch:0, training batch loss:0.9165
Training batch: 25 in epoch:0, training batch loss:0.8580


KeyboardInterrupt: 

In [25]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()
model = torch.load(model_file)
num_test = testX.shape[0]
test_num_batch = math.ceil(num_test / batch_size)
testPred = []
start_test = time.time()
for batch_idx in range(test_num_batch):
    start_idx = batch_idx * batch_size
    end_idx = min(num_test, (batch_idx + 1) * batch_size)
    X = testX[start_idx: end_idx].to(device)
    TE = testTE[start_idx: end_idx].to(device)
    pred_batch = model(X, TE)
    testPred.append(pred_batch.detach().clone())
    del X, TE, pred_batch
testPred = torch.cat(testPred, axis=0)
testPred = testPred* std + mean
max_length = int(math.log(data_args['Q'],2))
for i in range(max_length):
    if i!=0:
        start_idx = pow(2,i)
    else:
        start_idx = 0
    end_idx = pow(2,i+1)
    pred = testPred[:,start_idx:end_idx]
    label = testY[:,start_idx:end_idx]
    test_mae, test_rmse, test_mape = metric(pred, label.to(device))
    log_string(log, f'From step {start_idx} to step {end_idx}')
    log_string(log, f' Prediction performance:   MAE: {test_mae}, RMSE: {test_rmse}, MAPE: {test_mape}')
test_mae, test_rmse, test_mape = metric(testPred, testY.to(device))
log_string(log, 'During all steps')
log_string(log, f' Prediction performance:   MAE: {test_mae}, RMSE: {test_rmse}, MAPE: {test_mape}')
precision, recall, f1 = alarm_metric(testPred,testY,threshold)
log_string(log,  f' Alarm performance:   precision score: {precision}, recall score: {recall}, F1 score: {f1}')

From step 0 to step 2
 Prediction performance:   MAE: 0.3964328467845917, RMSE: 0.5046855211257935, MAPE: 0.11491795629262924
From step 2 to step 4
 Prediction performance:   MAE: 0.3468914031982422, RMSE: 0.4548645615577698, MAPE: 0.10042881220579147
From step 4 to step 8
 Prediction performance:   MAE: 0.38741934299468994, RMSE: 0.5100828409194946, MAPE: 0.11252658814191818
During all steps
 Prediction performance:   MAE: 0.37954074144363403, RMSE: 0.49547410011291504, MAPE: 0.11009998619556427


NameError: name 'alarm_metric' is not defined

In [9]:
print(testPred.shape)

torch.Size([5217, 8, 12])


In [10]:
model= torch.load(model_file)
num_train, _, num_vertex = trainX.shape
num_val = valX.shape[0]
num_test = testX.shape[0]
train_num_batch = math.ceil(num_train / batch_size)
val_num_batch = math.ceil(num_val / batch_size)
test_num_batch = math.ceil(num_test / batch_size)

# test model
log_string(log, '**** testing model ****')
log_string(log, 'loading model from %s' % model_file)
model = torch.load(model_file)
log_string(log, 'model restored!')
log_string(log, 'evaluating...')

with torch.no_grad():
    trainPred = []
    for batch_idx in range(train_num_batch):
        start_idx = batch_idx * batch_size
        end_idx = min(num_train, (batch_idx + 1) * batch_size)
        X = trainX[start_idx: end_idx]
        TE = trainTE[start_idx: end_idx]
        pred_batch = model(X, TE)
        trainPred.append(pred_batch.detach().clone())
        del X, TE, pred_batch
    trainPred = torch.from_numpy(np.concatenate(trainPred, axis=0))
    trainPred = trainPred * std + mean

    valPred = []
    for batch_idx in range(val_num_batch):
        start_idx = batch_idx * batch_size
        end_idx = min(num_val, (batch_idx + 1) * batch_size)
        X = valX[start_idx: end_idx]
        TE = valTE[start_idx: end_idx]
        pred_batch = model(X, TE)
        valPred.append(pred_batch.detach().clone())
        del X, TE, pred_batch
    valPred = torch.from_numpy(np.concatenate(valPred, axis=0))
    valPred = valPred * std + mean

    testPred = []
    start_test = time.time()
    for batch_idx in range(test_num_batch):
        start_idx = batch_idx * batch_size
        end_idx = min(num_test, (batch_idx + 1) * batch_size)
        X = testX[start_idx: end_idx]
        TE = testTE[start_idx: end_idx]
        pred_batch = model(X, TE)
        testPred.append(pred_batch.detach().clone())
        del X, TE, pred_batch
    testPred = torch.from_numpy(np.concatenate(testPred, axis=0))
    testPred = testPred* std + mean
end_test = time.time()
train_mae, train_rmse, train_mape = metric(trainPred, trainY)
val_mae, val_rmse, val_mape = metric(valPred, valY)
test_mae, test_rmse, test_mape = metric(testPred, testY)
log_string(log, 'testing time: %.1fs' % (end_test - start_test))
log_string(log, '                MAE\t\tRMSE\t\tMAPE')
log_string(log, 'train            %.2f\t\t%.2f\t\t%.2f%%' %
           (train_mae, train_rmse, train_mape * 100))
log_string(log, 'val              %.2f\t\t%.2f\t\t%.2f%%' %
           (val_mae, val_rmse, val_mape * 100))
log_string(log, 'test             %.2f\t\t%.2f\t\t%.2f%%' %
           (test_mae, test_rmse, test_mape * 100))
log_string(log, 'performance in each prediction step')
MAE, RMSE, MAPE = [], [], []
for step in range(num_pred):
    mae, rmse, mape = metric(testPred[:, step], testY[:, step])
    MAE.append(mae)
    RMSE.append(rmse)
    MAPE.append(mape)
    log_string(log, 'step: %02d         %.2f\t\t%.2f\t\t%.2f%%' %
               (step + 1, mae, rmse, mape * 100))
average_mae = np.mean(MAE)
average_rmse = np.mean(RMSE)
average_mape = np.mean(MAPE)
log_string(
    log, 'average:         %.2f\t\t%.2f\t\t%.2f%%' %
         (average_mae, average_rmse, average_mape * 100))



NameError: name 'log' is not defined